In [2]:
import pandas as pd

data = pd.read_csv('all_glucose_prediction_results.csv')
data

,Category,Start Date,Previous Lower Bound,Actual Glucose Change,Previous Upper Bound,Actual Variance,Prediction Status,Predicted Glucose Change,Predicted Variance
0,Lows,2023-12-04,0.461684,0.610000,0.758316,0.148316,True,0.610000,0.148316
1,Lows,2023-12-05,0.461684,0.572222,0.758316,0.153889,True,0.610000,0.148316
2,Lows,2023-12-06,0.418333,0.570588,0.726111,0.158456,True,0.572222,0.153889
3,Lows,2023-12-07,0.412132,0.461111,0.729044,0.121340,True,0.570588,0.158456
4,Lows,2023-12-08,0.339771,0.457895,0.582451,0.112573,True,0.461111,0.121340
...,...,...,...,...,...,...,...,...,...
199,Very Highs,2024-02-06,0.419466,0.733333,1.124284,0.348542,True,0.771875,0.352409
200,Very Highs,2024-02-07,0.384792,0.743333,1.081875,0.326678,True,0.733333,0.348542
201,Very Highs,2024-02-09,0.416655,0.806250,1.070011,0.359315,True,0.743333,0.326678
202,Very Highs,2024-02-10,0.446935,0.766667,1.165565,0.293542,True,0.806250,0.359315


In [3]:
category_stats = data.groupby('Category')[['Actual Glucose Change', 'Actual Variance']].agg(
    ['mean', 'median']
).reset_index()

category_order = ["Lows", "Mediums", "Highs", "Very Highs"]

category_stats_ordered = category_stats.set_index('Category').loc[category_order].reset_index()

print("Category Statistics in Custom Order (Lows, Mediums, Highs, Very Highs):")
print(category_stats_ordered)

Category Statistics in Custom Order (Lows, Mediums, Highs, Very Highs):
     Category Actual Glucose Change           Actual Variance          
                               mean    median            mean    median
0        Lows              0.413247  0.364286        0.102217  0.085330
1     Mediums              0.543849  0.543056        0.222196  0.224071
2       Highs              0.610227  0.600000        0.180053  0.182000
3  Very Highs              1.044959  0.982759        0.505549  0.512301


In [4]:
data_false_predictions = data[data['Prediction Status'] == False].copy()

if not data_false_predictions.empty:
    data_false_predictions.loc[:, 'Absolute Error Glucose Change'] = abs(data_false_predictions['Actual Glucose Change'] - data_false_predictions['Predicted Glucose Change'])
    data_false_predictions.loc[:, 'Absolute Error Variance Change'] = abs(data_false_predictions['Actual Variance'] - data_false_predictions['Predicted Variance'])

    mae_stats_glucose = data_false_predictions.groupby('Category')['Absolute Error Glucose Change'].agg(['mean', 'median']).reset_index()
    mae_stats_variance = data_false_predictions.groupby('Category')['Absolute Error Variance Change'].agg(['mean', 'median']).reset_index()

    available_categories_glucose = [cat for cat in category_order if cat in mae_stats_glucose['Category'].values]
    mae_stats_glucose_ordered = mae_stats_glucose.set_index('Category').loc[available_categories_glucose].reset_index()

    available_categories_variance = [cat for cat in category_order if cat in mae_stats_variance['Category'].values]
    mae_stats_variance_ordered = mae_stats_variance.set_index('Category').loc[available_categories_variance].reset_index()

    print("MAE Statistics for Glucose Change (False Predictions):")
    print(mae_stats_glucose_ordered)
    
    print("\nMAE Statistics for Variance Change (False Predictions):")
    print(mae_stats_variance_ordered)
else:
    print("No entries found with Prediction Status as False.")

MAE Statistics for Glucose Change (False Predictions):
     Category      mean    median
0        Lows  0.071687  0.063886
1       Highs  0.095833  0.095833
2  Very Highs  0.265546  0.265546

MAE Statistics for Variance Change (False Predictions):
     Category      mean    median
0        Lows  0.047709  0.052518
1       Highs  0.044736  0.044736
2  Very Highs  0.288054  0.288054
